In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras import backend as K

Using TensorFlow backend.


In [2]:
# allocates GPU memory based on runtime allocations. doesn't releases memory because of potential memory fragmentation
# without this option, getting images will not work
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

In [27]:
# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
nb_train_samples = 358
nb_validation_samples = 362
epochs = 50
batch_size = 16

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [17]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [18]:
# this is the augmentation configuration for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

In [28]:
# this is the augmentation configuration we will use for testing:
test_datagen = ImageDataGenerator(
    rescale=1./255)

In [20]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 361 images belonging to 2 classes.


In [33]:
validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 359 images belonging to 2 classes.


In [22]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size
)

model.save_weights('model.h5')

Epoch 1/50
22/22 [==============================] - 3s 154ms/step - loss: 0.6911 - acc: 0.6790 - val_loss: 0.7349 - val_acc: 0.4234
Epoch 2/50
22/22 [==============================] - 3s 145ms/step - loss: 0.6042 - acc: 0.7205 - val_loss: 0.7333 - val_acc: 0.4234
Epoch 3/50
22/22 [==============================] - 3s 143ms/step - loss: 0.5978 - acc: 0.7228 - val_loss: 1.6729 - val_acc: 0.4234
Epoch 4/50
22/22 [==============================] - 3s 136ms/step - loss: 0.6143 - acc: 0.6972 - val_loss: 0.7658 - val_acc: 0.4234
Epoch 5/50
22/22 [==============================] - 3s 130ms/step - loss: 0.5609 - acc: 0.7330 - val_loss: 0.7063 - val_acc: 0.4206
Epoch 6/50
22/22 [==============================] - 3s 131ms/step - loss: 0.5222 - acc: 0.7468 - val_loss: 0.7933 - val_acc: 0.4262
Epoch 7/50
22/22 [==============================] - 3s 126ms/step - loss: 0.5342 - acc: 0.7512 - val_loss: 0.7206 - val_acc: 0.5209
Epoch 8/50
22/22 [==============================] - 3s 127ms/step - loss: 0.

In [10]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 72, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 72, 72, 32)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 34, 34, 64)        18496     
__________

In [23]:
loaded_model = Sequential()

loaded_model.add(Conv2D(32, (3, 3), input_shape=input_shape))
loaded_model.add(Activation('relu'))
loaded_model.add(MaxPooling2D(pool_size=(2, 2)))

loaded_model.add(Conv2D(32, (3, 3)))
loaded_model.add(Activation('relu'))
loaded_model.add(MaxPooling2D(pool_size=(2, 2)))

loaded_model.add(Conv2D(64, (3, 3)))
loaded_model.add(Activation('relu'))
loaded_model.add(MaxPooling2D(pool_size=(2, 2)))

loaded_model.add(Flatten())
loaded_model.add(Dense(64))
loaded_model.add(Activation('relu'))
loaded_model.add(Dropout(0.5))
loaded_model.add(Dense(1))
loaded_model.add(Activation('sigmoid'))

loaded_model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

loaded_model.load_weights('model.h5')

In [32]:
prediction = loaded_model.predict_generator(validation_generator, verbose=1)

23/23 [==============================] - 1s 42ms/step


In [25]:
from sklearn.metrics import confusion_matrix
import numpy as np

y_true = np.array([0] * 207 + [1] * 152)
y_pred = prediction > 0.5

confusion_matrix(y_true, y_pred)

array([[ 67, 140],
       [ 53,  99]], dtype=int64)

In [34]:
prediction.shape

(359, 1)